The under codes are able to be implemented on R with rstan 

In [ ]:
naes<-read.csv("naes04.csv",header=T)
gayKnow<-table(naes$age,naes$gayKnowSomeone)
gayKnow<-data.frame(gayKnow)

no<-gayKnow[1:80,]
yes<-gayKnow[81:160,]

age<-as.numeric(as.character(no$Var1))
y<-no$Freq
n<-yes$Freq+no$Freq

phi<-function(x){
  phi<-NULL
  s<-seq(18,98,10)
  for (i in 1:length(s)){
    phi[i]<-exp(-(x-s[i])^2/(2*s[i]^2))
  }
  return(phi)
}

PHI<-matrix(1:length(age)*length(seq(18,98,10)),nrow=length(age),ncol=length(seq(18,98,10)))
for(i in 1:length(age)){
  PHI[i,]<-phi(age[i])
}

naes.dat<-list(N=length(y),M=9,n=n,y=y,X=age,nu=1.0,PHI=PHI)
naes.fit<-stan(file="naes04.stan",data=naes.dat,iter=2000,chains=4)


naes.ext<-extract(naes.fit,permuted=T)
w<-data.frame(naes.ext$beta)

p_rep<-matrix(1:dim(w)[1]*length(y),nrow=dim(w)[1],ncol=length(y))
y_rep<-matrix(1:dim(w)[1]*length(y),nrow=dim(w)[1],ncol=length(y))
for (i in 1:dim(w)[1]){
  for (j in 1:length(y)){
    p_rep[i,j]<-exp(PHI[j,]%*%as.numeric(w[i,]))/(1+exp(PHI[j,]%*%as.numeric(w[i,])))
    y_rep[i,j]<-rbinom(size=n[j],p=p_rep[i,j],n=1)
  }
  print(i)
}



g<-ggplot()
g<-g+geom_line(data=df,aes(x=df$age,y=df$apply.y_rep..2..quantile..2...))
g<-g+geom_line(data=df,aes(x=df$age,y=df$apply.y_rep..2..quantile..3...))
g<-g+geom_line(data=df,aes(x=df$age,y=df$apply.y_rep..2..quantile..4...))
g<-g+geom_point(data=df,aes(x=df$age,y=df$y))
plot(g)


In [ ]:
data {
  int<lower=0> N;
  int<lower=0> M;
  int<lower=0> n[N];
  int<lower=0> y[N];
  real<lower=0> X[N];
  real<lower=0> nu;
  matrix[N,M] PHI;
}

parameters {
  vector[M] beta;
  real<lower=0> alpha;
}


model{
  real p[N];
  
  for (i in 1:N){
    p[i] <- inv_logit(dot_product(PHI[i],beta));
    y[i] ~ binomial(n[i],p[i]);
  }
  beta ~ normal(0,sqrt(alpha));
  alpha ~ inv_gamma(nu/2,nu/2);
}

